In [ ]:
from tqdm import tqdm
import pickle
import pandas as pd
from pathlib import Path
import numpy as np
import scipy
import copy
import sys
import os
from typing import Optional, Union, Sequence, Mapping, List, Set
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import cm
from matplotlib.lines import Line2D
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

sys.path.append('/mnt/c/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/QiskitProjects/VariationalWavefunction/')

from quantum_kernel.code.utils import get_dataset
from quantum_kernel.code.visualization_utils import aggregate_shapes,filter_df
from quantum_kernel.code.kernel_measures.kernel_geometry import purity_average, purity_of_average,geometric_difference,geometric_distance
from quantum_kernel.code.kernel_measures.eigenvalue_scaling import get_eigenvalue_scaling,compute_dataframe_kernel_eigenvalues

font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 30}

rc('font', **font)

tqdm.pandas()
warnings.filterwarnings("ignore")


In [ ]:
# Get kernel results for the different ml models and dataset.d
dfs = {}
beta=1.0
alphas=[2.0]
gammas=[0.5]
folders = [
    ("/mnt/c/Users/lslat/Data/QK_project/results/Sparse_IQP/final_gennorm/alpha_test/beta{}/alpha{}/gamma{}/".format(beta,alphas[0],gammas[0]),'Sparse_IQP'),
]

cols_to_drop=['args', 'K_train_time', 'rdms', 
'samples', 'mu','sigma', 'outpath', 'projected']

def get_dfs_from_folder(folder,prefix,cols_to_drop):
    dfs={}
    dfs[prefix] = aggregate_shapes(folder,prefix,cols_to_drop=cols_to_drop)
    return dfs

dfs0=get_dfs_from_folder(folders[0][0],folders[0][1],cols_to_drop)
dfs_list=[dfs0]

In [ ]:
save_folder='/mnt/c/Users/lslat/Data/QK_project/results/figs/'

In [ ]:
fig, axs = plt.subplots(1,len(dfs_list),figsize=(30,10))
axs[0].set_ylabel('Max Kernel Eigenvalue')

gammas=[0.5]
for gamma,ax,dfs in zip(gammas,axs,dfs_list):
    iqp_df=dfs['Sparse_IQP']
    target_iqp_df=iqp_df[(iqp_df['density']!=0) & (iqp_df['beta']==beta)]
    xp=list(set(target_iqp_df.dataset_dim.values))
    yp=[np.mean(target_iqp_df[target_iqp_df.dataset_dim==x].apply(lambda x: max(x.kernel_eigenvalues),axis=1).values) for x in xp]
    yerr=[np.std(target_iqp_df[target_iqp_df.dataset_dim==x].apply(lambda x: max(x.kernel_eigenvalues),axis=1).values) for x in xp]

    ax.errorbar(xp,yp,yerr=yerr,fmt='o',ls='')
    ax.set_xlabel('Dataset Dimension')
    ax.set_title(rf'$\beta={beta}, \gamma_{0}={gamma}$',color='black')

    ax.plot(xp,len(xp)*[gamma],ls='-.',label=r'target $\gamma_{0}$')
ax.legend()
#fig.savefig(f'{save_folder}IQP_max_eigval_v_dim_fixed_beta{beta}.png')

In [ ]:

dataset_dims=[5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
seeds = set(target_iqp_df.seed.values)

folder="/mnt/c/Users/lslat/Data/QK_project/results/Sparse_IQP/final_gennorm/"

for gamma,dfs in zip(gammas,dfs_list):
    filepath=save_folder+f'geometric_difference_dict_beta{beta}gamma_{gamma}.p'
    if os.path.exists(filepath):
        with open(filepath, 'rb') as pickle_file:
            geo_diffs=pickle.load(pickle_file)
    else:
        geo_diffs={}

        iqp_df=dfs['Sparse_IQP']
        target_iqp_df=iqp_df[(iqp_df['density']!=0) & (iqp_df['beta']==beta)]
        for dim in dataset_dims:
            print(f'on dimension: {dim}')
            prefix=f"Sparse_IQP_dim_{dim}*_density_0_beta_{beta}"

            dfs=get_dfs_from_folder(folder,prefix,cols_to_drop)
            generic_ciqp_df=dfs[prefix]

            geo_diffs[str(dim)]=[]
            for seed in seeds:
                c_df=generic_ciqp_df[(generic_ciqp_df['seed']==seed)]
                q_df=target_iqp_df[(target_iqp_df['dataset_dim']==dim) & (target_iqp_df['seed']==seed)]
                Kq=q_df.qkern_matrix_train.values[0]
                Kcs=c_df.qkern_matrix_train.values
                sfs=c_df.scaling_factor.values
                geo_diffs[str(dim)]=geo_diffs[str(dim)]+[(sf,geometric_difference(Kq,Kc)) for sf,Kc in zip(sfs,Kcs)]
            geo_diffs[str(dim)]=np.array(geo_diffs[str(dim)])

        with open(filepath, 'wb') as f:
            try:
                pickle.dump(geo_diffs,f)
            except:
                print('Could not save geo_diffs dictionary')



In [ ]:
beta=1.0
gamma=0.5
filepath=save_folder+f'geometric_difference_dict_beta{beta}gamma_{gamma}.p'
if os.path.exists(filepath):
    with open(filepath, 'rb') as pickle_file:
        geo_diffs=pickle.load(pickle_file)
else:
    geo_diffs={}
colormap=cm.viridis
dataset_dims=[6,7,8,9,10,11,12,13,14,15,16,17,18,19]
fig, axs = plt.subplots(1,figsize=(15,10))
ax=axs
for dim in dataset_dims:
    color_index=int(256*dim/max(dataset_dims))-1
    color=colormap.colors[color_index]
    x=list(set(geo_diffs[str(dim)][:,0]))
    y=[np.mean(geo_diffs[str(dim)][geo_diffs[str(dim)][:,0]==a][:,1]) for a in x]
    yerr=[np.std(geo_diffs[str(dim)][geo_diffs[str(dim)][:,0]==a][:,1]) for a in x]
    ax.errorbar(x,y,yerr=yerr,color=color,label=dim,fmt='o',ls='')

ax.set_yscale('log')
axs.set_xscale('log')
ax.set_ylabel('g')
ax.set_xlabel('Classical Scaling Factor')
legend=plt.legend(ncol=2,bbox_to_anchor=(1,1),fontsize=18,title_fontsize=18)
legend.set_title('Dataset Dimension')
ax.set_title('g(classical IQP || quantum IQP)',pad=15)
fig.tight_layout()
#fig.savefig(f'{save_folder}IQP_gd_grid_search_beta{beta}_gamma{gamma}.png')

In [ ]:
colormap=cm.viridis
dataset_dims=[5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
fig, axs = plt.subplots(1,figsize=(15,10),sharey=True)

for i,gamma in enumerate(gammas):
    filepath=save_folder+f'geometric_difference_dict_beta{beta}gamma_{gamma}.p'
    if os.path.exists(filepath):
        with open(filepath, 'rb') as pickle_file:
            geo_diffs=pickle.load(pickle_file)
    else:
        continue
    ax=axs

    color_index=int(256*i/(len(gammas)))
    color=colormap.colors[color_index]
    
    ys=[]
    yerrs=[]
    for dim in dataset_dims:
        x=list(set(geo_diffs[str(dim)][:,0]))
        y=min([np.mean(geo_diffs[str(dim)][geo_diffs[str(dim)][:,0]==a][:,1]) for a in x])
        index = [np.mean(geo_diffs[str(dim)][geo_diffs[str(dim)][:,0]==a][:,1]) for a in x].index(y)
        yerr=[np.std(geo_diffs[str(dim)][geo_diffs[str(dim)][:,0]==a][:,1]) for a in x][index]

        ys.append(y)
        yerrs.append(yerr)
    ax.errorbar(dataset_dims,ys,yerr=yerrs,color=color,fmt='-o',label=rf'$\gamma_0={gamma}$')

    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_ylabel('minimium g')
    ax.set_xlabel('Dataset Dimension')

legend=plt.legend(ncol=2,bbox_to_anchor=(1,1),fontsize=18,title_fontsize=18)
legend.set_title('Dataset Dimension')

fig.suptitle(rf'minimum g(classical IQP || quantum IQP), $\beta={beta}$')

fig.tight_layout()
#fig.savefig(f'{save_folder}IQP_gd_decay_fixed_beta{beta}.png')